Import Necessary libraries

In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
image_dir = "C:/Users/DELL/null_class/traffic_model/pedestrain/PNGImages"
annotation_dir = "C:/Users/DELL/null_class/traffic_model/pedestrain/Annotation"

In [3]:
def parse_annotation(annotation_path):
    with open(annotation_path, 'r') as file:
        lines = file.readlines()
    
    bboxes = []
    for line in lines:
        if "Bounding box for object" in line:
            try:
                parts = line.split(':')[-1].strip().split('-')
                xmin, ymin = map(int, parts[0].strip('() ').split(','))
                xmax, ymax = map(int, parts[1].strip('() ').split(','))
                bboxes.append([xmin, ymin, xmax, ymax])
            except ValueError as e:
                print(f"Error parsing line '{line.strip()}': {e}")
    
    return bboxes

In [4]:
def load_data(image_dir, annotation_dir):
    images = []
    bboxes = []
    
    for image_file in os.listdir(image_dir):
        if image_file.endswith('.png'):
            image_path = os.path.join(image_dir, image_file)
            annotation_path = os.path.join(annotation_dir, os.path.splitext(image_file)[0] + '.txt')
            
            if os.path.exists(annotation_path):
                image = cv2.imread(image_path)
                image = cv2.resize(image, (224, 224))
                image = image / 255.0 
                bbox = parse_annotation(annotation_path)
                if bbox:  
                    images.append(image)
                    bboxes.append(bbox) 
    
    return np.array(images), bboxes
images, bboxes = load_data(image_dir, annotation_dir)

Splitting the Data

In [5]:
train_images, test_images, train_bboxes, test_bboxes = train_test_split(images, bboxes, test_size=0.2, random_state=42)

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense,Reshape, Concatenate

In [7]:
def create_yolo_model(input_shape, num_boxes, num_classes):
    inputs = Input(shape=input_shape)

    x = Conv2D(32, (3, 3), activation='relu')(inputs)
    x = MaxPooling2D((2, 2))(x)  
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x) 
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)  
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    
    output = Dense(num_boxes * (4 + num_classes), activation='linear')(x)
    output = Reshape((num_boxes, 4 + num_classes))(output)

    model = Model(inputs, output)
    return model

In [8]:
input_shape = (224, 224, 3)
num_boxes = 5  
num_classes = 1 

In [9]:
model = create_yolo_model(input_shape, num_boxes, num_classes)

In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [ ]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1024)           │    88,605,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 25)             │        25,625 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 5, 5)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 88,724,569 (338.46 MB)

 Trainable params: 88,724,569 (338.46 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
def data_generator(images, bboxes, batch_size, num_boxes):
    while True:
        for start in range(0, len(images), batch_size):
            end = min(start + batch_size, len(images))
            batch_images = images[start:end]
            batch_bboxes = bboxes[start:end]
            
            X = np.zeros((len(batch_images), 224, 224, 3))
            y = np.zeros((len(batch_images), num_boxes, 4 + num_classes))
            
            for i, img in enumerate(batch_images):
                X[i] = img
                for j, bbox in enumerate(batch_bboxes[i]):
                    if j < num_boxes:
                        y[i, j, :4] = bbox 
                        y[i, j, 4:] = 1  
            
            yield X, y

batch_size = 16
train_gen = data_generator(train_images, train_bboxes, batch_size, num_boxes)
val_gen = data_generator(test_images, test_bboxes, batch_size, num_boxes)

model.fit(train_gen, steps_per_epoch=len(train_images)//batch_size, epochs=200, validation_data=val_gen, validation_steps=len(test_images)//batch_size)


Epoch 1/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - loss: 9021.5527 - mae: 61.8855 - val_loss: 8491.9150 - val_mae: 62.6542
Epoch 2/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - loss: 5981.0669 - mae: 50.3644 - val_loss: 4265.2617 - val_mae: 40.6793
Epoch 3/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - loss: 4102.3882 - mae: 41.2758 - val_loss: 9808.8418 - val_mae: 58.3412
Epoch 4/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - loss: 3835.4077 - mae: 37.7447 - val_loss: 7193.1987 - val_mae: 51.4854
Epoch 5/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - loss: 4012.6213 - mae: 41.4169 - val_loss: 4024.1987 - val_mae: 38.1055
Epoch 6/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - loss: 3516.3745 - mae: 37.4272 - val_loss: 9744.9258 - val_mae: 57.0909
Epoch 7/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - loss: 3405.6016 - mae: 35.9746 - val_loss: 7256.0732 - val_mae: 49.3093
Epoch 8/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - loss: 3494.5725 - mae: 36.6789 - val_loss: 3889.0574 - val_mae: 38.1967
Epoch 9/200
8/8 

In [ ]:
def predict_pedestrians(image_path, model, num_boxes, confidence_threshold=0.5):
    image = cv2.imread(image_path)
    image_resized = cv2.resize(image, (224, 224))
    image_normalized = image_resized / 255.0
    image_input = np.expand_dims(image_normalized, axis=0)
    
    predictions = model.predict(image_input)[0]
    
    pedestrian_count = 0
    for i in range(num_boxes):
        xmin, ymin, xmax, ymax = predictions[i, :4]
        class_prob = predictions[i, 4]
        if class_prob > confidence_threshold:
            pedestrian_count += 1
    
    return pedestrian_count


In [ ]:
image_path = "C:/Users/DELL/null_class/traffic_model/pedestrain/PNGImages/FudanPed00054.png"
num_boxes = 5  

pedestrian_count = predict_pedestrians(image_path, model, num_boxes)

print(f'Number of pedestrians detected: {pedestrian_count}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
Number of pedestrians detected: 2


In [ ]:
model.save('pedestrian_detection.keras')